In [ ]:
import numpy as np
import pandas as pd
%cd '/home/amindo/Desktop/Question 3'

In [ ]:
df = pd.read_csv('mnist_train.csv')
df_test = pd.read_csv('mnist_test.csv')

aval az hame data ro scale mikonim

In [ ]:
#be df yek soton 1 baray mohase rahat tar b ezafe mikonim
def add_1_minmax_scaling(df,df_test):
    label=df['label']
    label_test = df_test['label']
    df.drop('label',axis = 1,inplace = True)
    df_test.drop('label',axis = 1,inplace = True)
    m = df.shape[0]
    m_test = df_test.shape[0]
    soton_1 = [1]*df.shape[0]
    soton_1_test = [1]*df.shape[0]
    columns =df.columns
    df = df.to_numpy()
    df_test = df_test.to_numpy()

    df1 = df.min(axis = 0).reshape(1,784)
    df1_test = df_test.min(axis = 0).reshape(1,784)

    df_min = np.ones((m,1)).dot(df1)
    df_test_min = np.ones((m,1)).dot(df1_test)
    
    df1 = df.max(axis  =0).reshape(1,784)
    df1_test = df_test.max(axis  =0).reshape(1,784)
    
    df_max = np.ones((m,1)).dot(df1)
    df_max_test = np.ones((m,1)).dot(df1_test)
    df = (df-df_min)/(df_max-df_min)
    df_test = (df-df_test_min)/(df_max_test-df_test_min)
    df = pd.DataFrame(df, columns=columns)
    df_test = pd.DataFrame(df_test, columns=columns)
    df  = df.fillna(0)
    df_test  = df_test.fillna(0)

    df['soton1'] = soton_1
    df_test['soton1'] = soton_1_test
    df['label'] = label
    df_test['label'] = label_test
    return(df,df_test)
df,df_test = add_1_minmax_scaling(df,df_test)

In [ ]:
#%store df
#%store df_test

In [1]:
#%store -r df
#%store -r df_test

cross validation bar hasb class 3 ba algorithm stochastic gradient descent

In [2]:
df.label.loc[(df.label != 3)] = -1
df.label.loc[(df.label == 3)] = 1
df_test.label.loc[(df_test.label != 3)] = -1
df_test.label.loc[(df_test.label == 3)] = 1

/home/amindo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
def error_cross(df,w):
    import pandas as pd 
    import numpy as np
    df2 = df.copy()
    m = df2.shape[0]
    n=df2.shape[1]
    df2['new'] = df2.iloc[:m,:n-1].dot(w)
    df2['predict'] = 0
    df2.predict.loc[(df2.new >= 0)] =1
    df2.predict.loc[(df2.new < 0)] =-1
    df2['accuracy'] = df2['label']- df2['predict']    
    k = ((df2['accuracy']==0).sum(axis = 0))/m
    return(k)

In [30]:
def cost_function_batch(df,w):
    df['loss'] = df.apply(lambda x:max(0,1-x['label']*(x.iloc[ :785].dot(w))),axis  = 1)
    return(df)

In [31]:
def batch(df,w,landa,lear_rate,epoch):
    df['new_label'] = df['label'].copy()
    import numpy as np
    m=df.shape[0]
    n = df.shape[1]
    for i in range(epoch):
        df = cost_function_batch(df,w)
        df['new_label'] = df['label'].copy()
        df['new_label'].loc[(df.loss == 0 )] =0
        df2= df.copy()
        df2= df.to_numpy()
        w = w-lear_rate*(landa*w + (-1/m)*((np.sum(np.multiply(df2,df2[:,n:n+1]),axis = 0))[:785].reshape(785,1)))
        df.drop(['loss','new_label'],axis = 1,inplace =True)
        print(i,landa)
    return(w)  

In [32]:
def k_fold_cross_val(df,k,landa):
    m = df.shape[0]
    s = int(m/k)
    accuracy_1 = []
    n = df.shape[1]
    for i in range(k):
        df = df.sample(frac = 1)
        df =df.reset_index()
        df.drop('index',axis = 1,inplace = True)
        w1 = np.random.normal(0,0.01,n-1).reshape(785,1)
        if i == k-1:
            df_test_1 = df.iloc[s*i:m,:]
        else:
            df_test_1 = df.iloc[s*i:s*(i+1),:]

        if i ==k-1:
            list1 =list(range(s*i,m))
        else:
            list1 =list(range(s*i,s*(i+1)))
        df_train = df.drop(index = list1)
        #y_test = Y_train[s*i:s*(i+1)]
        #y_train =Y_train.drop([list1],axis = 0)
        w = batch(df_train,w1,landa,0.01,15)
        accuracy_1.append(error_cross(df_test_1,w))
        #print(accuracy_1)
    return(sum(accuracy_1)/k)    

In [33]:
import numpy as np
import pandas as pd

In [36]:
landa_list = [1,10**-2,10**-4,10**-6,10**-8]
list2 = []
for i in landa_list:
    list2.append(k_fold_cross_val(df,10,i))
    print(f'for landa = {i} mean accuracy is {sum(list2)/len(list2)}')

for landa = 1 mean accuracy is 0.8880000000000001
for landa = 0.01 mean accuracy is 0.8980000000000001
for landa = 0.0001 mean accuracy is 0.8933333333333334
for landa = 1e-06 mean accuracy is 0.8955
for landa = 1e-16 mean accuracy is 0.8936
for landa = 0 mean accuracy is 0.895


hal ebteda batch gradient descent ro train mikonim ro kol data
baray sadegi mohasebe meghdar loss ro dar epoch hai 2,5,10,20,25 va 30 faghat mohasebe mikonim

In [ ]:
def cost_function_batch(df,w):
    df['loss'] = df.apply(lambda x:max(0,1-x['label']*(x.iloc[ :785].dot(w))),axis  = 1)
    return(df)

In [ ]:
def batch(df,w,landa,lear_rate,epoch):
    df['new_label'] = df['label'].copy()
    import numpy as np
    t = [0]*2
    m=df.shape[0]
    n = df.shape[1]
    z = np.zeros((785,6))
    for i in range(epoch):
        df = cost_function_batch(df,w)
        df['new_label'] = df['label'].copy()
        df['new_label'].loc[(df.loss == 0 )] =0
        df2= df.copy()
        df2= df.to_numpy()
        w = w-lear_rate*(landa*w + (-1/m)*((np.sum(np.multiply(df2,df2[:,n:n+1]),axis = 0))[:785].reshape(785,1)))
        df.drop(['loss','new_label'],axis = 1,inplace =True)
        if i+1 == 2:
            z[:,0:1] = w
        if i+1 ==5:
            z[:,1:2] = w
        if i+1 ==10:
            z[:,2:3] = w
        if i+1 ==20:
            z[:,3:4] = w
        if i+1 ==25:
            z[:,4:5] = w
        if i+1 ==30:
            z[:,5:6] = w
    t[0] = w 
    t[1] = z
    return(t)  

In [ ]:
def one_vs_all(df):
    import numpy as np
    import pandas as pd
    m = df.shape[0]
    n = df.shape[1]
    W = np.zeros([n-1,10])
    epoch_1 = []
    c= [0]*2
    for i in range(10):
    
        w = np.random.normal(0,0.01,n-1)
        w = pd.DataFrame(w)
        w = w.to_numpy()
        df2 = df.copy()
        df2.label.loc[(df2.label == i)] = 1
        df2.label.loc[(df2.label != i)] = -1
        a = batch(df2,w,10**-4,0.01,30)
        W[0:,i:i+1] =a[0]
        epoch_1.append(a[1])
        print('a')
    c=[W,epoch_1]
    return(c)

In [ ]:
c=one_vs_all(df)
W = c[0]
epoch_1_batch = c[1]

In [ ]:
%store df
%store W_batch
%store epoch_1_batch

In [ ]:
%store -r df
%store -r W_batch
%store -r epoch_1_batch

In [ ]:
def error(df,W_batch):
    import pandas as pd 
    import numpy as np
    k = [0]*2
    df2 = df.copy()
    m = df2.shape[0]
    n=df2.shape[1]
    df3 = df2.iloc[:m,:n-1].dot(W_batch)    
    df2['predict'] = df3.idxmax(axis =1)
    df2['accuracy'] = df2['label']- df2['predict']
    k[0] = ((df2['accuracy']==0).sum(axis = 0))/m
    k[1] = df2['predict']
    return(k)

accurasy ro baray df_test mohasebe mikonim


In [ ]:
accurasy_in_test = error(df_test,W_batch)[0]

In [ ]:
k =error(df,W)
error = k[0]
predict = k[1]

In [ ]:
def loss_function(df,W,predict,landa,epoch_1):
    import numpy as np
    m = df.shape[0]
    cos_fun_epoch =  []
    label = df['label']
    w = np.ones((785,1))
    for i in range(6):
            cost = 0
        for j in range(predict.shape[0]):
            w =epoch_1[predict[j]][:,i:i+1]
            if label[j]==predict[j]:
                cost = cost + (1/m)*(landa*(np.transpose(w).dot(w)[0,0]) + max(0,1-(np.sign(df.iloc[j:j+1,:785].dot(w).iloc[0,0]))*(df.iloc[j:j+1,:785].dot(w)).iloc[0,0]))
            else :
                cost = cost + (1/m)*(landa*(np.transpose(w).dot(w)[0,0]) + max(0,1+(np.sign(df.iloc[j:j+1,:785].dot(w).iloc[0,0]))*(df.iloc[j:j+1,:785].dot(w)).iloc[0,0]))
        cos_fun_epoch.append(cost)
    return(cos_fun_epoch)

In [ ]:
cos_fun_batch_epoch = loss_function(df,W,predict,10**-4,epoch_1)

In [ ]:
%store df
%store W

In [ ]:
%store -r df
%store -r W

In [ ]:
def loss_function(df,W,predict,landa,epoch_1):
    import numpy as np
    m = df.shape[0]
    cos_fun_epoch =  []
    label = df['label']
    w = np.ones((785,1))
    for i in range(6):
        cost = 0
        for j in range(predict.shape[0]):
            w =epoch_1[predict[j]][:,i:i+1]
            if label[j]==predict[j]:
                cost = cost + (1/m)*(landa*(np.transpose(w).dot(w)[0,0]) + max(0,1-(np.sign(df.iloc[j:j+1,:785].dot(w).iloc[0,0]))*(df.iloc[j:j+1,:785].dot(w)).iloc[0,0]))
            else :
                cost = cost + (1/m)*(landa*(np.transpose(w).dot(w)[0,0]) + max(0,1+(np.sign(df.iloc[j:j+1,:785].dot(w).iloc[0,0]))*(df.iloc[j:j+1,:785].dot(w)).iloc[0,0]))
        cos_fun_epoch.append(cost)
    return(cos_fun_epoch)

In [ ]:
def cost_function_batch(df,w):
    df['loss'] = df.apply(lambda x:max(0,1-x['label']*(x.iloc[ :785].dot(w))),axis  = 1)
    return(df)

In [ ]:
agar epoch ra baray stochastich 100 begirim baray rahat ta shadan mohasebe loss ra faghat baray zaraieb 10 mohasebeh mikonim

In [ ]:
def svm_gradient_decsent(df2,w,epoch,landa,lear_rate):
    import pandas as pd
    import numpy as np
    m = df2.shape[0]
    n = df2.shape[1]
    cost_functions_stochastic = [0]*epoch
    X = [0]*n
    new = pd.DataFrame(X,columns = ['new'])
   # global list1
    for j in range(epoch):
        if i+1 == 2:
            z[:,0:1] = w
        if i+1 ==5:
            z[:,1:2] = w
        if i+1 ==10:
            z[:,2:3] = w
        if i+1 ==20:
            z[:,3:4] = w
        if i+1 ==25:
            z[:,4:5] = w
        if i+1 ==30:
            z[:,5:6] = w
        np.random.shuffle(df2)
        for i in range(m):
            if (1-df2[i,n-1]*((df2[i:i+1,0:n-1]).dot(w)) <= 0):
                w = w-(landa/2)*w
            else :
                w = w-lear_rate*((landa/2)*w -(df2[i,n-1]*(df2[i:i+1,0:n-1].transpose())))
#        cost_functions_stochastic[j]= stochastic_cost(w,landa,m,n,new,df)
#    list1 = [w,cost_functions_stochastic]
    return(w)

In [ ]:
import numpy as np
import pandas as pd
def one_vs_all(df):
    m = df.shape[0]
    n = df.shape[1]
    W = np.zeros([n-1,10])
    for i in range(10):
        w = np.random.normal(0,0.01,n-1)
        w = pd.DataFrame(w)
        w = w.to_numpy()
        df2 = df.copy()
        df2.label.loc[(df2.label == i)] = 1
        df2.label.loc[(df2.label != i)] = -1
        
        df2 =df2.to_numpy()
        W[0:,i:i+1] = svm_gradient_decsent(df2,w,5,10**-4,0.01)
        print('a')
    return(W)

In [ ]:
W = one_vs_all(df)

In [ ]:
%store W
%store df

In [ ]:
%store -r W
%store -r df

In [ ]:
def error(df,W_stochastic):
    import pandas as pd 
    import numpy as np
    df2 = df.copy()
    m = df2.shape[0]
    n=df2.shape[1]
    df3 = df2.iloc[:m,:n-1].dot(W_stochastic)    
    df2['pridect'] = df3.idxmax(axis =1)
    df2['accuracy'] = df2['label']- df2['pridect']
    return(((df2['accuracy']==0).sum(axis = 0))/m)

In [ ]:
error(df,W)